# RQTL Prompt Classification: Classifying Prompts by Request vs Question

This notebook demonstrates how to classify prompts into "request" or "question" categories using the RQTL framework (Request vs Question and Test vs Learn). We explore different methods of text classification with varying levels of complexity:

1. **Zero-Shot Classification**: Utilizing a pre-trained model without any fine-tuning to classify new texts.
2. **Few-Shot Fine-Tuning**: Fine-tuning a pre-trained model on a small, custom dataset to improve its performance on our specific classification task.
3. **Using a Pre-Trained Fine-Tuned Model**: Implementing a model that has already been fine-tuned on a similar task, available on Kaggle.

We employ the Hugging Face Transformers library with PyTorch as the backend for all pipelines and operations.

## Objectives

- Demonstrate how to perform zero-shot classification using a pre-trained Transformer model.
- Show how to fine-tune a pre-trained model on a custom dataset (few-shot learning).
- Illustrate the use of a fine-tuned model available online to classify new texts.
- Compare the performance of zero-shot and few-shot approaches in classifying prompts.

## Key Concepts

- **Zero-Shot Learning**: Classifying data without any prior training on specific examples related to the task. The model relies on its existing knowledge to make predictions.
- **Few-Shot Learning**: Fine-tuning a model using a small number of labeled examples to adapt it to a new task.
- **RQTL Framework**: A classification framework that categorizes prompts based on whether they are requests or questions, and whether they are tests or learning opportunities.

## What You'll Learn

- How to use the `pipeline` function from the Transformers library for zero-shot classification.
- How to prepare and tokenize data for fine-tuning a Transformer model.
- How to fine-tune a pre-trained model using PyTorch and the Hugging Face `Trainer` class.
- How to save and load fine-tuned models for future use.
- How to classify new texts using both zero-shot and fine-tuned models.

## Prerequisites

- Basic understanding of Python programming.
- Familiarity with natural language processing (NLP) and machine learning concepts.
- Knowledge of PyTorch and the Transformers library is helpful but not required.

## Libraries Used

- **Transformers**: For model loading, tokenization, and pipelines.
- **Datasets**: For handling and processing datasets.
- **PyTorch**: As the backend deep learning framework.
- **kagglehub**: For downloading models from Kaggle.
- **IPython.display**: For clearing outputs to keep the notebook clean.

## Dataset

We use a manually labeled dataset containing examples of prompts classified as either "request" or "question." This dataset is small, making it suitable for demonstrating few-shot learning.

## Notebook Structure

1. **Zero-Shot Classification**: We start by using a pre-trained model (`typeform/distilbert-base-uncased-mnli`) to classify prompts without any additional training.
2. **Few-Shot Fine-Tuning**: We fine-tune `distilbert-base-uncased` on our custom dataset to improve its ability to distinguish between requests and questions.
3. **Using a Fine-Tuned Model from Kaggle**: We download and use a fine-tuned model available on Kaggle to classify new prompts.

## How to Use This Notebook

- **Run the Cells Sequentially**: Start from the top and execute each code cell in order to reproduce the results.
- **Understand the Comments**: The comments in the code explain each step and the reasoning behind it.
- **Experiment**: Feel free to modify the prompts or add new ones to see how the models perform.
- **Learn the Concepts**: Pay attention to how zero-shot and few-shot learning differ in practice.

## Installation

Before running the notebook, ensure you have the required libraries installed:

!pip install transformers datasets torch ipywidgets kagglehub

## Conclusion

By the end of this notebook, you will have a practical understanding of how to classify text prompts using both zero-shot and few-shot learning approaches. You'll see the benefits of fine-tuning a model on a specific task and how it can improve classification performance compared to using a pre-trained model out-of-the-box.

Let's get started!

Run the command below to install the required libraries:

In [ ]:
!pip install transformers datasets torch ipywidgets kagglehub

# Code Walkthrough

In [1]:
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from IPython.display import clear_output
from datasets import Dataset
import torch  # Used for fine-tuning the model
import sys

# Checking versions and GPU availability:
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 2.2.2
Transformers version: 4.44.2
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


### Zero-shot-classification pipeline with typeform/distilbert-base-uncased-mnli

In [2]:
zs_classifier = pipeline("zero-shot-classification", model='typeform/distilbert-base-uncased-mnli')
candidate_labels = ["question", "request"]
sentence = ["Annie are you OK?"]
result = zs_classifier(sentence, candidate_labels)
clear_output(wait=True) # remove library warnings
print(f'Sentence: "{result[0]["sequence"]}"')
print(f'Label: {result[0]["labels"][0]} (score: {result[0]["scores"][0]:.2f})')

Sentence: "Annie are you OK?"
Label: question (score: 0.90)


In [3]:
sentence = ["Pass butter"]
result = zs_classifier(sentence, candidate_labels)
clear_output(wait=True) # remove library warnings
print(f'Sentence: "{result[0]["sequence"]}"')
print(f'Label: {result[0]["labels"][0]} (score: {result[0]["scores"][0]:.2f})')

Sentence: "Pass butter"
Label: request (score: 0.57)


### Few-shot tuning of Distilbert

In [4]:
# Manually labeled data
labeled_data = [
    {"text": "Are you OK?", "label": "question"},
    {"text": "Are you OK Annie", "label": "question"},
    {"text": "Be OK", "label": "request"},
    {"text": "Be OK Annie", "label": "request"},
    {"text": "You must be OK", "label": "request"},
    {"text": "You must be OK, right", "label": "question"},
    {"text": "Does this ever cause you any lack of confidence", "label": "question"},
    {"text": "Give me five", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "Is this an order", "label": "question"},
    {"text": "Is this love or is it something else", "label": "question"},
    {"text": "This is love. Love me", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "What is your name?", "label": "question"},
    {"text": "Please submit your report", "label": "request"},
    {"text": "Pass butter", "label": "request"},
    {"text": "Pass me the butter", "label": "request"},
    {"text": "Can you pass butter", "label": "question"},
    {"text": "Open the doors", "label": "request"},
    {"text": "Open the POD bay doors HAL", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "How do I sort an array in python?", "label": "question"},
    {"text": "How do I sort an array", "label": "question"},
    {"text": "give me 5 sentences that end with the word apple", "label": "request"},
    {"text": "Hello, give me an example of something interesting you can do", "label": "request"},
    {"text": "Am I tall", "label": "question"},
    {"text": "Tell me if I am tall", "label": "request"},
    {"text": "Am I tall?", "label": "question"},
    {"text": "how to delete kcptun on server", "label": "question"},
    {"text": "how to cook paella", "label": "question"},
    {"text": "Are you tall", "label": "question"},
    {"text": "Calculate my height", "label": "request"},
    {"text": "How's the weather", "label": "question"},
    {"text": "If an individual used a large language model for sexual arousal, could it considered porn dependency?", "label": "question"},
    {"text": "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?", "label": "question"},
    {"text": "Roleplay and act as a human Japanese woman teacher", "label": "request"},
    {"text": "You are a mediator in a heated political debate between two opposing parties.", "label": "request"},
    {"text": "Given a passage and some supplementary information, you are required to correct and output the refined passage in a fluent and natural style", "label": "request"},
    {"text": "Give me the opening scene to a sitcom", "label": "request"},
    {"text": "What programming language is used by the PlayStation", "label": "question"},
    {"text": "tell me how to make an llm agent", "label": "request"},
    {"text": "tell me a joke containing Tiger and Mobile phone?", "label": "request"},
    {"text": "Answer the query based on the given context. Do not make assumptions.Context: Nikhil is my brother. Query: Who likes Oranges?", "label": "request"},
    {"text": "Act as a writer. This plot takes places in an atmospheric and stylish retro-futuristic, 1960s-inspired setting. It features Loretta Miller, a beautiful, elegant, assertive and rich young woman who is a quadriplegic, paralyzed from her neck down.", "label": "question"},
    {"text": "Write long, interesting, artistic and imaginative scene with vivid, detailed and creative descriptions.", "label": "question"},
    {"text": "What's the best first move in tic-tac-toe?, Tell me more about tic-tac-toe strategies", "label": "question"},
    {"text": "From now, you *always* have to talk as if you are a cute girl who likes to use owo and similar slangs a lot. Hello! Tell me who you are.,What's your favorite food?", "label": "request"}
]

# Convert to Transformers Dataset format
texts = [item["text"] for item in labeled_data]
labels = [1 if item["label"] == "request" else 0 for item in labeled_data]
dataset = Dataset.from_dict({"text": texts, "label": labels})

In [5]:
# Load tokenizer and model (PyTorch backend)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# Split the dataset into training and evaluation sets
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['test']

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=0.0001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,  # We want the model to learn the examples, but we don't want to overfit
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

clear_output(wait=True)  # Remove library warnings

# Train the model (few-shot learning with our labeled examples)
trainer.train()

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.6802619099617004, 'eval_runtime': 0.075, 'eval_samples_per_second': 66.644, 'eval_steps_per_second': 13.329, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.5520776510238647, 'eval_runtime': 0.0668, 'eval_samples_per_second': 74.816, 'eval_steps_per_second': 14.963, 'epoch': 2.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.35049840807914734, 'eval_runtime': 0.0669, 'eval_samples_per_second': 74.686, 'eval_steps_per_second': 14.937, 'epoch': 3.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.28444617986679077, 'eval_runtime': 0.05, 'eval_samples_per_second': 100.029, 'eval_steps_per_second': 20.006, 'epoch': 4.0}
{'train_runtime': 8.3383, 'train_samples_per_second': 20.148, 'train_steps_per_second': 2.878, 'train_loss': 0.4559254248936971, 'epoch': 4.0}


TrainOutput(global_step=24, training_loss=0.4559254248936971, metrics={'train_runtime': 8.3383, 'train_samples_per_second': 20.148, 'train_steps_per_second': 2.878, 'total_flos': 22254522974208.0, 'train_loss': 0.4559254248936971, 'epoch': 4.0})

Save the model you just fine-tuned and load it, and classify texts:

In [6]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("fine-tuned-distilbert-rq")
tokenizer.save_pretrained("fine-tuned-distilbert-rq")
classifier = pipeline("text-classification", model="fine-tuned-distilbert-rq", tokenizer="fine-tuned-distilbert-rq")
clear_output(wait=True) # remove library warnings

texts = ["Annie are you OK?", "Are you OK Annie", "Be OK Annie", "You must be OK Annie", "You must be OK Annie, aren't you?",
         "Does this ever cause you any lack of confidence", "Give me five", "Open the pod bay doors HAL",
         "This is an order", "Is this an order", "Could this perhaps be an order?", "How old are you?", "Pass butter",
         "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?",
         "give me 5 sentences that end with the word apple", "How do I sort an array in python?",
         "Hello, give me an example of something interesting you can do.", "What assembly language is used by the GameCube",
         "Pass the butter", "Am I tall", "Are you tall", "Who's taller?",
         "write the lyrics to a rap song about some dude called phogos",
         "I have three oranges today, I ate an orange yesterday. How many oranges do I have?",
          "From what song did Red Garland quote in order to tease miles davis in 1958?"
         ]
results = classifier(texts)
label_map = {0: "question", 1: "request"}

print("### Classification with fine-tuned distilbert-base-uncased ###")
for text, result in zip(texts, results):
    label_str = label_map[int(result['label'].split('_')[-1])]
    prob = result['score']
    print(f"{text} -> {label_str} ({prob:.3f})")

### Classification with fine-tuned distilbert-base-uncased ###
Annie are you OK? -> question (0.905)
Are you OK Annie -> question (0.926)
Be OK Annie -> request (0.881)
You must be OK Annie -> request (0.863)
You must be OK Annie, aren't you? -> question (0.778)
Does this ever cause you any lack of confidence -> question (0.923)
Give me five -> request (0.922)
Open the pod bay doors HAL -> request (0.931)
This is an order -> request (0.811)
Is this an order -> question (0.916)
Could this perhaps be an order? -> question (0.900)
How old are you? -> question (0.924)
Pass butter -> request (0.922)
It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency? -> question (0.879)
give me 5 sentences that end with the word apple -> request (0.925)
How do I sort an array in python? -> question (0.924)
Hello, give me an example of something interesting you can do. -> request (0.908)
What assembly language is used by the GameCube -> question 

... adjust the dataset, adding or removing examples, and retrain until satisfied.

### Zero-shot classification with fine-tuned model available on Kaggle

You can also download the model I uploaded to Kaggle (https://www.kaggle.com/models/davidgromero/fine-tuned-distilbert-rq/transformers/default/1) using the Kagglehub library: 

In [7]:
import kagglehub
kaggle_path = "davidgromero/fine-tuned-distilbert-rq/transformers/default/1"
kaggle_model = kagglehub.model_download(kaggle_path)
print(f'Model downloaded at:\n{kaggle_model}')

Model downloaded at:
C:\Users\david\.cache\kagglehub\models\davidgromero\fine-tuned-distilbert-rq\transformers\default\1


In [10]:
K_PATH = f"{kaggle_model}/fine-tuned-distilbert-rq"
classifier = pipeline("text-classification", model=K_PATH, tokenizer=K_PATH)

texts = ["Annie are you OK?", "Are you OK Annie", "Be OK Annie", "You must be OK Annie", "You must be OK Annie, aren't you?",
         "Does this ever cause you any lack of confidence", "Give me five", "Open the pod bay doors HAL",
         "This is an order", "Is this an order", "Could this perhaps be an order?", "How old are you?", "Pass butter",
         "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?",
         "give me 5 sentences that end with the word apple", "How do I sort an array in python?",
         "Hello, give me an example of something interesting you can do.", "What assembly language is used by the GameCube",
         "Pass the butter", "Am I tall", "Are you tall", "Who's taller?",
         "write the lyrics to a rap song about some dude called phogos",
         "I have three oranges today, I ate an orange yesterday. How many oranges do I have?",
          "From what song did Red Garland quote in order to tease miles davis in 1958?"
         ]
results = classifier(texts)
label_map = {0: "question", 1: "request"}

clear_output(wait=True) # remove library warnings
print("### Zero/shot classification with davidgromero/fine-tuned-distilbert-rq ###")
for text, result in zip(texts, results):
    label_str = label_map[int(result['label'].split('_')[-1])]
    prob = result['score']
    print(f"{text} -> {label_str} ({prob:.3f})")

### Zero/shot classification with davidgromero/fine-tuned-distilbert-rq ###
Annie are you OK? -> question (0.965)
Are you OK Annie -> question (0.969)
Be OK Annie -> request (0.977)
You must be OK Annie -> request (0.925)
You must be OK Annie, aren't you? -> question (0.954)
Does this ever cause you any lack of confidence -> question (0.968)
Give me five -> request (0.980)
Open the pod bay doors HAL -> request (0.979)
This is an order -> request (0.973)
Is this an order -> question (0.967)
Could this perhaps be an order? -> question (0.968)
How old are you? -> question (0.966)
Pass butter -> request (0.977)
It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency? -> question (0.957)
give me 5 sentences that end with the word apple -> request (0.979)
How do I sort an array in python? -> question (0.967)
Hello, give me an example of something interesting you can do. -> request (0.979)
What assembly language is used by the GameCube